# Power Consumption Analysis

This notebook analyzes power consumption metrics from CSV data and correlates them with LLM benchmark request timings.

## Import Required Libraries

In [1]:
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from datetime import datetime
import json

## Load and Prepare CSV Data

In [ ]:
# Open CSV file and load data into DataFrame
data = pd.read_csv('.\\benchmark_results\\local.CSV', encoding='latin-1', decimal='.', thousands=',')
# Remove leading/trailing whitespace and special characters (like BOM) from column names
data.columns = data.columns.str.strip().str.replace('\ufeff', '', regex=False)

# Show the first few rows of the DataFrame
print(data.head())
print(f"\nColumn names: {data.columns[:5].tolist()}")  # Print first 5 column names to verify

## Filter Relevant Columns

In [ ]:
# Filter columns to keep only the relevant metrics
columns_to_keep = [
    'Potenza totale CPU [W]',
    "CPU Package Power [W]",
    
    'Potenza Core IA [W]',
    "IA Cores Power [W]",
    
    'VR VCC Corrente (SVID IOUT) [A]',
    "VR VCC Current (SVID IOUT) [A]"
    
    'GPU Potenza [W]',
    "GPU Power [W]",
    
    'IGPU Potenza [W]',
    "IGPU Power [W]"
    
    'Potenza DRAM totale [W]',
    'Total DRAM Power [W]',
    
    'Consumo energetico resto del chip [W]',
    'Rest-of-Chip Power [W]'
]

# Keep only columns that exist in the DataFrame
existing_columns = [col for col in columns_to_keep if col in data.columns]
data_filtered = data[existing_columns]

## Convert to Numeric and Handle Missing Values

In [ ]:
# Convert all columns to numeric values, handling errors
for col in data_filtered.columns:
    data_filtered[col] = pd.to_numeric(data_filtered[col], errors='coerce')

# Replace NaN values with 0
data_filtered = data_filtered.fillna(0)

print(data_filtered.head())
print(data_filtered.dtypes)

## Add Timestamp Column

In [ ]:
# Convert the first two time-related columns to datetime
# Combine Date and Time columns to create a datetime column
data_filtered['timestamp'] = pd.to_datetime(
    data['Date'] + ' ' + data['Time'],
    format='%d.%m.%Y %H:%M:%S.%f',
    errors='coerce'
)

print(f"\nCSV time range (corrected): {data_filtered['timestamp'].min()} to {data_filtered['timestamp'].max()}")

## Load JSON Benchmark Data

In [ ]:
# Load JSON benchmark data
json_file = '.\\benchmark_results\\benchmark_qwen_qwen3-4b-thinking-2507_20260126_004413.json'
with open(json_file, 'r', encoding='utf-8') as f:
    benchmark_data = json.load(f)

# Extract request data and timestamps from JSON
request_events = []
if 'results' in benchmark_data:
    for entry in benchmark_data['results']:
        request_events.append({
            'timestamp_send': pd.to_datetime(entry['timestamp_send']),
            'timestamp_response': pd.to_datetime(entry['timestamp_response']),
            'total_tokens': entry.get('total_tokens', 0),
            'elapsed_time': entry.get('elapsed_time_seconds', 0),
            'size_category': entry.get('size_category', 'unknown')
        })

# Convert to DataFrame
request_df = pd.DataFrame(request_events)

if len(request_df) > 0:
    print(f"\nJSON time range: {request_df['timestamp_send'].min()} to {request_df['timestamp_send'].max()}")
    print(f"Total requests: {len(request_df)}")

## Filter CSV Data to Match Request Timeline

In [ ]:
if len(request_df) > 0:
    # Get the first timestamp_send and last timestamp_response from JSON
    first_json_timestamp = request_df['timestamp_send'].min()
    last_json_timestamp = request_df['timestamp_response'].max()
    print(f"\nFirst timestamp_send in JSON: {first_json_timestamp}")
    print(f"Last timestamp_response in JSON: {last_json_timestamp}")
    
    # Filter CSV data to show only from first request send to last response
    data_filtered = data_filtered[
        (data_filtered['timestamp'] >= first_json_timestamp) & 
        (data_filtered['timestamp'] <= last_json_timestamp)
    ].copy()
    print(f"CSV data filtered to {len(data_filtered)} rows (from first request to last response)")

## Match Requests with CSV Timestamps

In [ ]:
if len(request_df) > 0:
    # For each request, find the closest CSV measurement
    request_df['csv_index'] = request_df['timestamp_send'].apply(
        lambda req_time: (data_filtered['timestamp'] - req_time).abs().idxmin() 
        if pd.notna(data_filtered['timestamp']).any() else None
    )
    
    # Get the closest timestamp for each request
    request_df['closest_csv_time'] = request_df['csv_index'].apply(
        lambda idx: data_filtered.loc[idx, 'timestamp'] if idx is not None else None
    )
    
    request_df['time_diff_seconds'] = (request_df['timestamp_send'] - request_df['closest_csv_time']).dt.total_seconds()
    
    print(f"\nTime difference statistics (seconds):")
    print(request_df['time_diff_seconds'].describe())

## Visualize Power Metrics with Request Time Periods

In [ ]:
# Define colors for each category
category_colors = {
    'short': 'green',
    'medium': 'skyblue',
    'long': 'red',
}

# Plot power metrics with request time bands
for column in existing_columns:
    plt.figure(figsize=(14, 6))
    
    # Plot power data vs timestamp
    ax1 = plt.gca()
    ax1.plot(data_filtered['timestamp'], data_filtered[column], label=column, color='blue', linewidth=1.5)
    ax1.set_ylabel('Power (W)', color='blue', fontsize=12)
    ax1.set_xlabel('Time', fontsize=12)
    ax1.tick_params(axis='y', labelcolor='blue')
    ax1.grid(True, alpha=0.3)
    
    # Add time bands for each request
    if len(request_df) > 0:
        # Track which categories have been added to legend
        added_to_legend = set()
        
        for idx, row in request_df.iterrows():
            category = row['size_category']
            color = category_colors.get(category, 'lightgray')
            label = f'{category.capitalize()} Request' if category not in added_to_legend else None
            
            # Draw vertical span from timestamp_send to timestamp_response
            ax1.axvspan(row['timestamp_send'], row['timestamp_response'], 
                       alpha=0.3, color=color, label=label)
            
            if label:
                added_to_legend.add(category)
    
    plt.title(f'{column} with LLM Request Time Periods', fontsize=14, fontweight='bold')
    ax1.legend(loc='lower left')
    
    # Format x-axis for better readability
    plt.gcf().autofmt_xdate()
    plt.tight_layout()

plt.show()

In [ ]:
# Define a comprehensive statistics calculation function
def calculate_power_statistics(data, requests, power_columns=None, group_by=None, filter_by=None):
    """
    Calculate power consumption statistics with flexible grouping and filtering.
    
    Parameters:
    - data: DataFrame with power metrics and timestamps
    - requests: DataFrame with request information
    - power_columns: List of power columns to analyze (default: all existing_columns)
    - group_by: List of grouping criteria ('model', 'size_category', 'timestamp_hour')
    - filter_by: Dict with column name and values to filter (e.g., {'size_category': ['short', 'medium']})
    
    Returns:
    - Dictionary with statistics for each group
    """
    
    if power_columns is None:
        power_columns = existing_columns
    
    if group_by is None:
        group_by = ['size_category']
    
    # Add csv_index and power data to requests
    working_df = requests.copy()
    for col in power_columns:
        working_df[col] = working_df['csv_index'].apply(
            lambda idx: data.loc[idx, col] if idx is not None and col in data.columns else 0
        )
    
    # Calculate energy consumption during each request
    working_df['duration_seconds'] = (working_df['timestamp_response'] - working_df['timestamp_send']).dt.total_seconds()
    for col in power_columns:
        working_df[f'{col}_energy_Wh'] = (working_df[col] * working_df['duration_seconds']) / 3600
    
    # Apply filters if provided
    if filter_by:
        for col, values in filter_by.items():
            working_df = working_df[working_df[col].isin(values)]
    
    # Calculate statistics
    results = {}
    
    # Group by specified criteria
    if group_by:
        grouped = working_df.groupby(group_by)
        for group_name, group_data in grouped:
            group_key = str(group_name)
            results[group_key] = {}
            
            for col in power_columns:
                results[group_key][col] = {
                    'avg_power_W': group_data[col].mean(),
                    'total_energy_Wh': group_data[f'{col}_energy_Wh'].sum(),
                    'max_power_W': group_data[col].max(),
                    'min_power_W': group_data[col].min(),
                    'std_dev': group_data[col].std(),
                    'count': len(group_data)
                }
    else:
        # Overall statistics
        results['overall'] = {}
        for col in power_columns:
            results['overall'][col] = {
                'avg_power_W': working_df[col].mean(),
                'total_energy_Wh': working_df[f'{col}_energy_Wh'].sum(),
                'max_power_W': working_df[col].max(),
                'min_power_W': working_df[col].min(),
                'std_dev': working_df[col].std(),
                'count': len(working_df)
            }
    
    return results, working_df

# Calculate correlation between power consumption and request characteristics
def calculate_correlations(working_df, power_columns=None):
    """
    Calculate correlations between power metrics and request characteristics.
    """
    if power_columns is None:
        power_columns = existing_columns
    
    correlations = {}
    
    for col in power_columns:
        correlations[col] = {
            'vs_total_tokens': working_df[col].corr(working_df['total_tokens']),
            'vs_elapsed_time': working_df[col].corr(working_df['elapsed_time']),
            'vs_duration_seconds': working_df[col].corr(working_df['duration_seconds'])
        }
    
    return correlations

# Display results in a readable format
def display_statistics(stats_dict, title="Power Consumption Statistics"):
    """Display statistics in a formatted table."""
    print(f"\n{'='*80}")
    print(f"{title}")
    print(f"{'='*80}\n")
    
    for group, metrics in stats_dict.items():
        print(f"\n{group}:")
        print("-" * 80)
        for power_col, stats in metrics.items():
            print(f"\n  {power_col}:")
            for metric, value in stats.items():
                if isinstance(value, float):
                    print(f"    {metric}: {value:.2f}")
                else:
                    print(f"    {metric}: {value}")

# Example usage - Calculate statistics grouped by size_category
stats_by_category, df_with_energy = calculate_power_statistics(
    data_filtered, 
    request_df,
    power_columns=existing_columns,
    group_by=['size_category']
)

display_statistics(stats_by_category, "Statistics by Request Size Category")

# Calculate overall correlations
correlations = calculate_correlations(df_with_energy, existing_columns)
print("\n\nCorrelations with Request Characteristics:")
for col, corr_dict in correlations.items():
    print(f"\n{col}:")
    for metric, value in corr_dict.items():
        print(f"  {metric}: {value:.4f}")